In [ ]:
!pip install --upgrade opencv-python transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
!pip install -U git+https://github.com/huggingface/diffusers.git
!pip install -U git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-13vj2813
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-13vj2813
  Resolved https://github.com/huggingface/diffusers.git to commit 0c1e63bd11a5746db8933111a962854fa9b36582
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.31.0.dev0-py3-none-any.whl size=2658778 sha256=44dbbcaa79e15e86d02fa14a75e8d400f231a2f99a311c8223dfee68448fdb8e
  Stored in directory: /tmp/pip-ephem-wheel-cache-aqq5zkyl/wheels/4d/b7/a8/6f9549ceec5daad78675b857ac57d697c387062506520a7b50
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.30.0
    Uninstalling diffusers-0.30.0:
      Successfully uninstalled diffusers-0.30.0


  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-qqq_g10k
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-qqq_g10k
  Resolved https://github.com/huggingface/accelerate.git to commit a452327e8e04b20779882dc491e00de602d554cb
  Installing build dependencies ... canceledERROR: Operation cancelled by user


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing

In [ ]:
import gc
import torch
import gradio as gr
from diffusers import CogVideoXPipeline, CogVideoXDDIMScheduler
from diffusers.utils import export_to_video

def reset_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_accumulated_memory_stats()
    torch.cuda.reset_peak_memory_stats()

def print_memory():
    memory = round(torch.cuda.memory_allocated() / 1024**3, 2)
    max_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
    max_reserved = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
    print(f"{memory=} GB")
    print(f"{max_memory=} GB")
    print(f"{max_reserved=} GB")

pipe = CogVideoXPipeline.from_pretrained("THUDM/CogVideoX-2b", torch_dtype=torch.float16)
pipe.scheduler = CogVideoXDDIMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_tiling()

def generate_video(prompt, num_frames, progress=gr.Progress(track_tqdm=True)):
    reset_memory()

    with torch.cuda.amp.autocast():
        video = pipe(
            prompt=prompt,
            num_frames=num_frames,
            guidance_scale=6,
            num_inference_steps=30,
            generator=torch.Generator().manual_seed(42),
            callback=lambda i, t, latents: progress(i, t.shape[0])
        ).frames[0]

    print_memory()
    output_path = "output_video.mp4"
    export_to_video(video, output_path, fps=8)
    return output_path

with gr.Blocks() as demo:
    gr.Markdown(
        """
        # CogVideoX Video Generation

        Enter a detailed prompt to generate a video. CogVideoX works best with large, descriptive prompts.
        Avoid short or simple prompts for better results.

        Note: Video generation might take a few minutes. Please be patient.
        """
    )

    with gr.Row():
        prompt_input = gr.Textbox(label="Enter your detailed video prompt", lines=5)
        num_frames = gr.Dropdown(choices=[24, 48], value=24, label="Number of frames")

    generate_button = gr.Button("Generate Video")
    video_output = gr.Video(label="Generated Video")

    generate_button.click(
        generate_video,
        inputs=[prompt_input, num_frames],
        outputs=video_output
    )

demo.launch(share=True, debug=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The config attributes {'mid_block_add_attention': True} were passed to AutoencoderKLCogVideoX, but are not expected and will be ignored. Please verify your config.json configuration file.


AttributeError: 'AutoencoderKLCogVideoX' object has no attribute 'enable_tiling'

## You do not have to run the cell below, it's just testing code if you don't want the gradio UI ;)

In [ ]:
import gc
import torch
from diffusers import CogVideoXPipeline, CogVideoXDDIMScheduler
from diffusers.utils import export_to_video

def reset_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_accumulated_memory_stats()
    torch.cuda.reset_peak_memory_stats()

def print_memory():
    memory = round(torch.cuda.memory_allocated() / 1024**3, 2)
    max_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
    max_reserved = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
    print(f"{memory=} GB")
    print(f"{max_memory=} GB")
    print(f"{max_reserved=} GB")

prompt = (
    "A cat walking through the forest sees a bear and gets scared and starts running through a muddy path that is along the middle of the forest , the path is in the middle and at both sides are trees, the bear is chasing the cat, we see the point of view of the cat as the camera is facing the cat and in the background we can see an angry bear chasing down the cat along the muddy path in the middle of the forest.	"
)

pipe = CogVideoXPipeline.from_pretrained("THUDM/CogVideoX-2b", torch_dtype=torch.float16)
pipe.scheduler = CogVideoXDDIMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

pipe.enable_sequential_cpu_offload()  # Changed from enable_model_cpu_offload
pipe.vae.enable_tiling()

reset_memory()

# Reduce video parameters
with torch.cuda.amp.autocast():  # Enable automatic mixed precision
    video = pipe(
        prompt=prompt,
        num_frames=48,  # Reduced from 48
        guidance_scale=6,
        num_inference_steps=30,  # Reduced from 50
        generator=torch.Generator().manual_seed(42)
    ).frames[0]

print_memory()
export_to_video(video, "output_tiling.mp4", fps=8)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The config attributes {'mid_block_add_attention': True, 'sample_size': 256} were passed to AutoencoderKLCogVideoX, but are not expected and will be ignored. Please verify your config.json configuration file.


  0%|          | 0/30 [00:00<?, ?it/s]

memory=0.07 GB
max_memory=8.72 GB
max_reserved=10.48 GB


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


'output_tiling.mp4'